**Author:** Revekka Gersgovich

**Purpose:** Explore the data with children's mental health stigma or adult mental health stigma modules

**Date:** Nov 29, 2025

In [1]:
import os
import os.path as path
import pandas as pd
import numpy as np
import glob
import narwhals
import pyreadstat

In [2]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Documents/Filling_system/Academic/Taste-Based_Discrimination") # Change this directory to run from your computer
assert os.path.exists(parent_dir), "parent_dir does not exist"
os.chdir(parent_dir)

raw_data_dir = path.join(parent_dir, "1_data", "1_raw")
assert os.path.exists(raw_data_dir), "raw_data_dir does not exist"

intermediate_data_dir = path.join(parent_dir, "1_data", "2_intermediate")
assert os.path.exists(intermediate_data_dir), "intermediate_data_dir does not exist"

# Loading Data

In [3]:
df = pd.read_csv(path.join(intermediate_data_dir, "gss_cleaned_96_02_06_18_24.csv"))